In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("Data Frames").getOrCreate()

all_RDD = sc.wholeTextFiles("FileStore/tables/city_data/")

x1 = all_RDD.flatMapValues(lambda t: t.split("\r\n"))
rdd = x1.map(lambda t : (t[0].replace("dbfs:/FileStore/tables/city_data/","").replace(".txt",""),t[1].split(" ")[0],t[1].split(" ")[1]))
rdd.collect()

In [3]:
df = spark.createDataFrame(rdd, schema=['City','Month','Revenue'])

In [4]:
rdd2 = rdd.map(lambda t : (t[0] if "_" not in t[0] else t[0][:-2],t[1],t[2]))
df_bis = spark.createDataFrame(rdd2, schema=['City','Month','Revenue'])
df_bis.show()

In [5]:
# Average monthly income of the shop in France

df.agg(F.avg('Revenue').alias("Avg income in France")).show()

In [6]:
# Average monthly income of the shop in each city

df_bis.groupBy('City').agg(F.avg('Revenue').alias("Avg income in each city")).show()

In [7]:
#Total revenue per city per year

df_bis.groupBy('City').agg(F.sum('Revenue').alias("Tot revenue per city")).show()

In [8]:
#Total revenue per store per year

df.groupBy('City').agg(F.sum('Revenue').alias("Total revenue per store")).show()

In [9]:
#The store that achieves the best performance in each month

df.createOrReplaceTempView("data")

spark.sql("""
    SELECT City, Month, Revenue
    FROM data JOIN (SELECT Month m1, max(Revenue) m2 FROM data GROUP BY Month) temp on Month = temp.m1 AND Revenue = temp.m2
""").show()